In [22]:
# Install required packages
%pip install langchain-tavily certifi

# For pretty printing and visualization
%pip install rich pandas


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [23]:
import os
import ssl
from typing import Any, Dict, List

import certifi
from langchain_tavily import TavilyCrawl
from rich.console import Console
from rich.panel import Panel
from rich.table import Table

# Configure SSL context
ssl_context = ssl.create_default_context(cafile=certifi.where())
os.environ["SSL_CERT_FILE"] = certifi.where()
os.environ["REQUESTS_CA_BUNDLE"] = certifi.where()

# Initialize rich console for pretty printing
console = Console()

print("✅ All imports successful!")


✅ All imports successful!


In [24]:
# Set directly (uncomment and add your key)
# tavily_api_key = "your_tavily_api_key_here"

os.environ["TAVILY_API_KEY"] = "your_tavily_api_key_here"


In [25]:
# Initialize TavilyCrawl
tavily_crawl = TavilyCrawl()

print("✅ TavilyCrawl initialized successfully!")


✅ TavilyCrawl initialized successfully!


In [26]:
# Basic TavilyCrawl without instructions
target_url = "https://python.langchain.com/"

console.print(Panel.fit(
    f"🎯 **Target**: {target_url}\n📋 **Instructions**: None (baseline crawl)",
    title="Basic Crawl Configuration",
    border_style="yellow"
))

console.print("🔄 Running TavilyCrawl without instructions...", style="bold yellow")

# Basic crawl without instructions
basic_result = tavily_crawl.invoke({
    "url": target_url,
    "max_depth": 3,
    "extract_depth": "advanced"
})

basic_results = basic_result.get("results", [])
console.print(f"✅ Basic crawl completed! Found {len(basic_results)} pages", style="bold green")

# Show what we got without instructions
console.print(f"\n📊 **Without Instructions**: TavilyCrawl returned {len(basic_results)} pages", style="bold yellow")
console.print("   📄 Mix of all content types (chains, prompts, agents, guides, etc.)")
console.print("   🔍 No filtering - everything from the crawled sections")
console.print("   ⚠️  Requires manual work to find what you actually need")


╭───────── Basic Crawl Configuration ──────────╮
│ 🎯 **Target**: https://python.langchain.com/ │
│ 📋 **Instructions**: None (baseline crawl)   │
╰──────────────────────────────────────────────╯

🔄 Running TavilyCrawl without instructions...

✅ Basic crawl completed! Found 0 pages

📊 **Without Instructions**: TavilyCrawl returned 0 pages

📄 Mix of all content types (chains, prompts, agents, guides, etc.)

🔍 No filtering - everything from the crawled sections

⚠️  Requires manual work to find what you actually need

In [27]:
# Let's look at what the basic crawl found
console.print("\n📋 **Sample Results from Basic Crawl (no filtering):**\n", style="bold yellow")

for i, result in enumerate(basic_results[:3], 1):  # Show first 3 results
    title = result.get("title", "Untitled")
    url = result.get("url", "No URL")
    content = result.get("raw_content", "No content")[:150] + "..."
    
    panel_content = f"""
🔗 **URL**: {url}

📖 **Content Preview**:
{content}
    """.strip()
    
    console.print(Panel(
        panel_content,
        title=f"📄 {i}. {title}",
        border_style="yellow"
    ))
    print()

console.print(f"... and {len(basic_results) - 3} more mixed results", style="italic yellow")
console.print("🔍 **Notice**: These are mixed content types - not specifically about agents!", style="bold yellow")


📋 **Sample Results from Basic Crawl (no filtering):**

... and -3 more mixed results

🔍 **Notice**: These are mixed content types - not specifically about agents!

In [28]:
# Now let's add intelligent instructions to the same target
instructions = "Find and extract content from pages specifically related to agents"

console.print(Panel.fit(
    f"🎯 **Target**: {target_url} (same as Demo 1)\n📋 **Instructions**: {instructions}",
    title="Intelligent Crawl Configuration", 
    border_style="green"
))


╭──────────────────────────── Intelligent Crawl Configuration ────────────────────────────╮
│ 🎯 **Target**: https://python.langchain.com/ (same as Demo 1)                           │
│ 📋 **Instructions**: Find and extract content from pages specifically related to agents │
╰─────────────────────────────────────────────────────────────────────────────────────────╯

In [29]:
# Execute the intelligent crawl - this demonstrates TavilyCrawl's power!
console.print("🚀 Starting intelligent crawl with instructions...", style="bold blue")
console.print("Watch how instructions transform the results!", style="italic")

# Use TavilyCrawl with instructions
crawl_result = tavily_crawl.invoke({
    "url": target_url,
    "instructions": instructions,
    "max_depth": 3,
    "extract_depth": "advanced"
})

console.print("✅ Intelligent crawl completed successfully!", style="bold green")

# Show the power of intelligent filtering
results = crawl_result.get("results", [])

# Compare with previous demo
console.print(f"\n🎯 **The Magic of Instructions:**", style="bold blue")
console.print(f"   📊 Demo 1 (no instructions): {len(basic_results)} mixed pages")
console.print(f"   🎯 Demo 2 (with instructions): {len(results)} targeted agent pages")
console.print(f"   ⚡ **Intelligent filtering eliminated {len(basic_results) - len(results)} irrelevant pages!**", style="bold green")


🚀 Starting intelligent crawl with instructions...

Watch how instructions transform the results!

✅ Intelligent crawl completed successfully!

🎯 **The Magic of Instructions:**

📊 Demo 1 (no instructions): 0 mixed pages

🎯 Demo 2 (with instructions): 0 targeted agent pages

   ⚡ **Intelligent filtering eliminated 0 irrelevant pages!**

In [30]:
# Display the agent documentation TavilyCrawl found
console.print("\n🎯 **LangChain Agent Documentation Found by TavilyCrawl:**\n", style="bold green")

for i, result in enumerate(results, 1):
    title = result.get("title", "Untitled")
    url = result.get("url", "No URL")
    content = result.get("raw_content", "No content")[:200] + "..."
    
    panel_content = f"""
🔗 **URL**: {url}

📖 **Content Preview**:
{content}
    """.strip()
    
    console.print(Panel(
        panel_content,
        title=f"📑 {i}. {title}",
        border_style="green"
    ))
    print()

# Show the comparison table
comparison_table = Table(title="⚡ Crawling Approaches Comparison")
comparison_table.add_column("Approach", style="cyan", no_wrap=True)
comparison_table.add_column("Time Required", style="yellow")
comparison_table.add_column("Expertise Needed", style="red")
comparison_table.add_column("Flexibility", style="blue")
comparison_table.add_column("Precision", style="green")

comparison_table.add_row(
    "Manual Crawling",
    "3-5 hours",
    "High (coding, filtering)",
    "Low",
    "Variable"
)

comparison_table.add_row(
    "TavilyMap + Extract",
    "30-60 minutes",
    "High (caveats, complexity)",
    "High",
    "Good (with expertise)"
)

comparison_table.add_row(
    "🤖 TavilyCrawl",
    "2-5 minutes",
    "Minimal (just instructions)",
    "High",
    "High (AI-powered)"
)

console.print(comparison_table)

console.print("\n🎉 **Why TavilyCrawl is the Best of Both Worlds:**", style="bold magenta")
console.print("   🚀 **Speed**: 60-100x faster than manual crawling")
console.print("   🧠 **Simplicity**: No expertise needed - just natural language instructions")
console.print("   🎯 **Flexibility**: High flexibility without the complexity of Map+Extract")
console.print("   ⚡ **Precision**: AI-powered filtering with no caveats or manual work")
console.print("   🔧 **Ready-to-Use**: Immediate integration into your applications")

console.print("\n💡 **TavilyCrawl = TavilyMap + TavilyExtract + AI Intelligence - Complexity!**", style="bold blue")


🎯 **LangChain Agent Documentation Found by TavilyCrawl:**

                                     ⚡ Crawling Approaches Comparison                                     
┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Approach            ┃ Time Required ┃ Expertise Needed            ┃ Flexibility ┃ Precision             ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━┩
│ Manual Crawling     │ 3-5 hours     │ High (coding, filtering)    │ Low         │ Variable              │
│ TavilyMap + Extract │ 30-60 minutes │ High (caveats, complexity)  │ High        │ Good (with expertise) │
│ 🤖 TavilyCrawl      │ 2-5 minutes   │ Minimal (just instructions) │ High        │ High (AI-powered)     │
└─────────────────────┴───────────────┴─────────────────────────────┴─────────────┴───────────────────────┘

🎉 **Why TavilyCrawl is the Best of Both Worlds:**

🚀 **Speed**: 60-100x faster than manual crawling

🧠 **Simplicity**: No expertise needed - just natural language instructions

🎯 **Flexibility**: High flexibility without the complexity of Map+Extract

⚡ **Precision**: AI-powered filtering with no caveats or manual work

🔧 **Ready-to-Use**: Immediate integration into your applications

💡 **TavilyCrawl = TavilyMap + TavilyExtract + AI Intelligence - Complexity!**